# Homework 2: Rating Prediction from Review Text

In [3]:
YELP_TRAIN_DIR = r"C:\Users\Nebiyou Hailemariam\Desktop\development\Machine-Learning-with-Graphs\Assignment 2\data\yelp_reviews_train.json"
YELP_DEV_DIR = r"C:\Users\Nebiyou Hailemariam\Desktop\development\Machine-Learning-with-Graphs\Assignment 2\data\yelp_reviews_dev.json"
YELP_TEST_DIR = r"C:\Users\Nebiyou Hailemariam\Desktop\development\Machine-Learning-with-Graphs\Assignment 2\data\yelp_reviews_test.json"
STOP_WORD_DIR = r"C:\Users\Nebiyou Hailemariam\Desktop\development\Machine-Learning-with-Graphs\Assignment 2\data\stopword.list"

In [4]:
import json

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            json_data = [json.loads(line) for line in file]
            return json_data
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error: {e}")
        return None

def get_stop_words(file_path):
    stop_words = {}
    with open(file_path, 'r') as file:
        for line_number, word in enumerate(file, start=1):
            word = word.strip()  # Remove leading and trailing whitespaces
            stop_words[word] = True
    return stop_words

train_data = load_json_file(YELP_TRAIN_DIR)

stop_words = get_stop_words(STOP_WORD_DIR)

In [5]:
def star_distribution(reviews):
    stars = {}

    for review in reviews:
        stars[review["stars"]] = 1 + stars.get(review["stars"], 0)

    total = sum(stars.values())
    distribution = [{"stars": key, "dist": stars[key] / total} for key in sorted(stars.keys())]

    return stars, distribution

star_distribution(train_data)

({5: 463084, 2: 112547, 4: 373469, 3: 178215, 1: 128038},
 [{'stars': 1, 'dist': 0.10199362251095907},
  {'stars': 2, 'dist': 0.0896536671358574},
  {'stars': 3, 'dist': 0.14196405313883823},
  {'stars': 4, 'dist': 0.29750118094273087},
  {'stars': 5, 'dist': 0.36888747627161445}])

In [4]:
import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def preprocess_text(text):
    sentence = remove_punctuation(text.lower()).split(" ")
    return [word for word in sentence if word != '' and word not in stop_words]

In [5]:
def build_vocab_ctf(reviews):
    vocab = {}
    for review in reviews:
         words = preprocess_text(review['text'])

         for word in words:
            vocab[word] = 1 + vocab.get(word, 0)

    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    top_2000_vocab = dict(sorted_vocab[:2000])

    return top_2000_vocab

def build_vocab_df(reviews):
    vocab = {}
    for review in reviews:
        document_frequency  = {}
        words = preprocess_text(review['text'])

        for word in words:
            document_frequency[word] = 1

        for word in document_frequency.keys():
            vocab[word] = document_frequency[word] + vocab.get(word, 0)

    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    top_2000_vocab = dict(sorted_vocab[:2000])

    return top_2000_vocab

In [65]:
import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def preprocess_text(text):
    sentence = remove_punctuation(text.lower()).split(" ")
    return [
        word
        for word in sentence
        if word != ""
        and word not in stop_words
    ]

def build_vocab(reviews):
    vocab = {}
    for review in reviews:
         words = preprocess_text(review['text'].lower())

         for word in words:
            vocab[word] = 1 + vocab.get(word, 0)

    sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
    top_2000_vocab = dict(sorted_vocab[:2000])

    return top_2000_vocab


In [66]:
vocab = build_vocab(train_data)

AttributeError: 'dict' object has no attribute 'lower'

In [64]:
build_vocab(["re2...svi!!!ews ", "re2.sviews_", 'goOd', 'gOO00d'])

{'good': 1}

In [36]:
vocab['good']

717886

In [21]:
from collections import Counter
import re

with open(YELP_TRAIN_DIR, 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]
    data = [(item["text"], item["stars"] ) for item in data]
    with open ("data/stopword.list", "r") as myfile:
        stopwords = myfile.read().splitlines()
    stopwords_punkt  =  stopwords + list(string.punctuation)

    ## preprocess training data
    X_train_preprocessed = []
    Y_train = []
    c_docs = Counter()
    c_stars = Counter()
    # stopat = 0
    pattern = re.compile(r'[\W\d_]')
    for doc, label in data :
        # Remove punctuation, digits, and filter out empty tokens using regular expressions
        intermediate = [pattern.sub('', word) for word in(doc.lower().split())]
        # Remove empty tokens
        intermediate = [word for word in intermediate if word and word not in stopwords_punkt ]
        X_train_preprocessed.append(' '.join(intermediate))
        Y_train.append(label)
        c_docs.update(intermediate)
        c_stars.update([label])
 

print(c_docs.most_common(10))

[('good', 742864), ('place', 716168), ('food', 691619), ('great', 585195), ('like', 546189), ('just', 521374), ('time', 442499), ('service', 431404), ('really', 387363), ('nice', 271367)]


In [18]:
vocab

{'good': 717886,
 'place': 705901,
 'food': 668636,
 'great': 568882,
 'like': 541612,
 'just': 516881,
 'time': 432549,
 'service': 402826,
 'really': 384510,
 'nice': 265274,
 'got': 261062,
 'best': 238447,
 'love': 229190,
 'little': 227892,
 'ordered': 204995,
 'pretty': 199383,
 'people': 198149,
 'restaurant': 198006,
 'try': 196273,
 'went': 193582,
 'came': 190580,
 'order': 187794,
 'vegas': 180203,
 'staff': 179415,
 'chicken': 178574,
 'menu': 177425,
 'friendly': 173657,
 'know': 171489,
 'come': 170514,
 'make': 168679,
 'better': 168597,
 'bar': 164783,
 'going': 164438,
 'definitely': 162996,
 'way': 162936,
 'night': 160758,
 'think': 157914,
 'right': 151120,
 'want': 150352,
 'room': 150148,
 'delicious': 149973,
 'experience': 145380,
 'say': 145111,
 'amazing': 140805,
 'day': 140686,
 'new': 139579,
 'said': 138826,
 'wait': 136549,
 'eat': 136407,
 'sure': 133668,
 'cheese': 129400,
 'pizza': 128827,
 'bit': 126081,
 'fresh': 125663,
 '2': 124423,
 'sauce': 12340

In [12]:
import numpy as np

def preprocess_dataset_ctf(reviews, vocab):
    vocab_index = list(vocab.keys())
    features = np.zeros((len(reviews), 2000), dtype=np.int32)
    targets = []

    for review_idx in range(len(reviews)):

        sentence = preprocess_text(reviews[review_idx]['text'])
        targets.append(reviews[review_idx]["stars"])

        reviews[review_idx] = None
        
        for word in sentence:
            if word in vocab and features[review_idx][vocab_index.index(word)] == 0:
                num_of_word_count = sentence.count(word)
                features[review_idx][vocab_index.index(word)] = num_of_word_count

    return features, targets

def preprocess_dataset_ctf_test(reviews, vocab):
    vocab_index = list(vocab.keys())
    features = np.zeros((len(reviews), 2000), dtype=np.int32)

    for review_idx in range(len(reviews)):
        sentence = preprocess_text(reviews[review_idx]['text'])

        reviews[review_idx] = None
        
        for word in sentence:
            if word in vocab and features[review_idx][vocab_index.index(word)] == 0:
                num_of_word_count = sentence.count(word)
                features[review_idx][vocab_index.index(word)] = num_of_word_count

    return features

In [13]:
import numpy as np

def preprocess_dataset_df(reviews, vocab):
    vocab_index = list(vocab.keys())
    features = np.zeros((len(reviews), 2000), dtype=np.int32)
    targets = []

    for review_idx in range(len(reviews)):

        sentence = preprocess_text(reviews[review_idx]['text'])
        targets.append(reviews[review_idx]["stars"])

        reviews[review_idx] = None
        
        for word in sentence:
            if word in vocab:
                features[review_idx][vocab_index.index(word)] = vocab[word]

    return features, targets

def preprocess_dataset_df_test(reviews, vocab):
    vocab_index = list(vocab.keys())
    features = np.zeros((len(reviews), 2000), dtype=np.int32)

    for review_idx in range(len(reviews)):
        sentence = preprocess_text(reviews[review_idx]['text'])

        reviews[review_idx] = None
        
        for word in sentence:
            if word in vocab:
                features[review_idx][vocab_index.index(word)] = vocab[word]

    return features

### CTF

In [8]:
vocab = build_vocab_ctf(train_data)
train_features, train_targets = preprocess_dataset_ctf(train_data, vocab)
train_features, train_targets = np.array(train_features), np.array(train_targets)

### DF

In [14]:
vocab = build_vocab_df(train_data)
train_features, train_targets = preprocess_dataset_df(train_data, vocab)
train_features, train_targets = np.array(train_features), np.array(train_targets)

In [15]:
train_features, train_targets = np.array(train_features), np.array(train_targets)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets, test_size=0.000001, random_state=42)

## SVM

In [18]:
from sklearn.model_selection import train_test_split
from sklearn import svm

clf = svm.LinearSVC(penalty='l2', loss='squared_hinge', random_state=0)

clf.fit(X_train, y_train)

c:\Users\Nebiyou Hailemariam\Desktop\development\Machine-Learning-with-Graphs\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [16]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.574878919194494


# Logistic Regression

In [102]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def softmax(pred):
    return np.exp(pred) / np.sum(np.exp(pred), axis=0, keepdims=True)

class LogisticRegression:
    def __init__(self, dim, classes, lr, lambda_reg=1):
        self.lr = lr
        self.lambda_reg = lambda_reg
        self.classes = classes
        self.W = np.zeros((dim, self.classes))
        self.X = None

    def forward(self, X):
        self.X = X
        return softmax(np.matmul(self.W.T, X))

    def backward(self, output, label):
        dl_dw = np.matmul(label - output, self.X.T).T - self.lambda_reg * self.W
        self.W = self.W + self.lr * (dl_dw / self.X.shape[1])

In [ ]:
intercept_column = np.ones((len(train_data), 1))
X_with_intercept = np.hstack((intercept_column, train_features))
num_classes = len(np.unique(train_targets))
label = np.eye(num_classes)[np.array(train_targets)-1]

X_train, X_test, Y_train, Y_test = train_test_split(X_with_intercept, label, test_size=0.0001, random_state=42)

In [128]:
model = LogisticRegression(dim=X_with_intercept.shape[1], classes=num_classes, lr=0.1, lambda_reg=1)
batch_size = 512

epochs = 0
while epochs < 10:
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size].T
        Y_batch = Y_train[i:i+batch_size].T
        pred = model.forward(X_batch)
        model.backward(output=pred, label=Y_batch)
    epochs += 1
    print(f"Epoch #{epochs}")

Epoch #1
Epoch #2
Epoch #3
Epoch #4
Epoch #5
Epoch #6
Epoch #7
Epoch #8
Epoch #9
Epoch #10


: 

In [119]:
test_pred = np.argmax(model.forward(X_test.T), axis=0)
accuracy = np.mean(test_pred == np.argmax(Y_test.T, axis=0)) * 100
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 57.14%


In [120]:
def get_hard_prediction(model, test):
    return np.argmax(model.forward(test.T), axis=0)

def get_soft_prediction(model, test):
    return np.sum(model.forward(test.T) * np.array([[1], [2], [3], [4], [5]]), axis=0)

In [121]:
def merge_and_save_vectors(vector1, vector2, output_file_path):
    merged_vector = np.column_stack((vector1, vector2))
    np.savetxt(output_file_path, merged_vector, fmt=['%d', '%.3f'], delimiter=' ')

### Predict on Dev Data

In [ ]:
dev_data = load_json_file(YELP_DEV_DIR)
dev_features = preprocess_dataset_ctf_test(dev_data)
dev_features = np.array(dev_features)
intercept_column = np.ones((len(dev_features), 1))
dev_features = np.hstack((intercept_column, dev_features))

In [126]:
merge_and_save_vectors(get_hard_prediction(model, dev_features), get_soft_prediction(model, dev_features), "./dev-predictions.txt")

### Predict on Test Data

In [57]:
test_data = load_json_file(YELP_TEST_DIR)
test_features = preprocess_dataset_ctf_test(test_data)
test_features = np.array(test_features)
intercept_column = np.ones((len(test_features), 1))
test_features = np.hstack((intercept_column, test_features))

In [127]:
merge_and_save_vectors(get_hard_prediction(model, test_features), get_soft_prediction(model, test_features), "./test-predictions.txt")